In [94]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,AIMessagePromptTemplate
model = ChatOpenAI()
print(model.model_name)

gpt-3.5-turbo


In [2]:
# step one - Import Parser
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [3]:
# step two - format instructions
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [4]:
reply = "red, blue, green"

In [7]:
output_parser.parse(reply)

['red', 'blue', 'green']

In [10]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [24]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [25]:
model_request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [26]:
result = model(model_request)

In [27]:
result.content

'Loyal, playful, protective, obedient, energetic'

In [28]:
output_parser.parse(result.content)

['Loyal', 'playful', 'protective', 'obedient', 'energetic']

# when the model ddoes not parser correctly

In [30]:
model_request = chat_prompt.format_prompt(request="write me a blog post about dogs",
                          format_instructions=output_parser.get_format_instructions()).to_messages()
result = model(model_request)
result.content

"Dogs are often called man's best friend, and for good reason. They are loyal, loving, and endlessly devoted to their human companions. Here are some reasons why dogs are such amazing creatures:\n\n1. Loyalty: Dogs have an incredible sense of loyalty to their owners. They will stand by you through thick and thin, always ready to provide comfort and support.\n\n2. Unconditional love: Dogs love their owners unconditionally, never judging or holding grudges. They are always happy to see you, no matter what kind of day you've had.\n\n3. Playfulness: Dogs have an infectious sense of playfulness that can bring joy to even the dreariest of days. Whether they're chasing a ball, going for a walk, or just wagging their tails, dogs know how to have fun.\n\n4. Protection: Dogs are natural protectors, always on the lookout for any potential threats to their loved ones. Their loyalty and bravery make them excellent guard dogs and companions.\n\n5. Companionship: Dogs are excellent companions, provid

In [31]:
output_parser.parse(result.content)

["Dogs are often called man's best friend",
 'and for good reason. They are loyal',
 'loving',
 'and endlessly devoted to their human companions. Here are some reasons why dogs are such amazing creatures:',
 '1. Loyalty: Dogs have an incredible sense of loyalty to their owners. They will stand by you through thick and thin',
 'always ready to provide comfort and support.',
 '2. Unconditional love: Dogs love their owners unconditionally',
 'never judging or holding grudges. They are always happy to see you',
 "no matter what kind of day you've had.",
 "3. Playfulness: Dogs have an infectious sense of playfulness that can bring joy to even the dreariest of days. Whether they're chasing a ball",
 'going for a walk',
 'or just wagging their tails',
 'dogs know how to have fun.',
 '4. Protection: Dogs are natural protectors',
 'always on the lookout for any potential threats to their loved ones. Their loyalty and bravery make them excellent guard dogs and companions.',
 '5. Companionship: D

In [32]:
from langchain.output_parsers import DatetimeOutputParser

In [48]:
datetime_parser = DatetimeOutputParser()

In [49]:
datetime_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z\n\nReturn ONLY this string, no other words!"

In [95]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

# when get parser error, use it to refine the anwser
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")

In [96]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [97]:
model_request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                          format_instructions = datetime_parser.get_format_instructions()).to_messages()

In [98]:
result = model(model_request, temperature=0)

In [99]:
result.content

'1865-12-06T00:00:00.000000Z'

In [70]:
datetime_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

### fixing outpur parser

In [100]:
from langchain.output_parsers import OutputFixingParser

In [101]:
misformatted = result.content

In [109]:
new_parser = OutputFixingParser.from_llm(parser=datetime_parser,llm=model)
new_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z\n\nReturn ONLY this string, no other words!"

In [110]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)